In [1]:
import numpy as np
import scipy.integrate
import symbolic_pofk.linear
import symbolic_pofk.syrenhalofit as syrenhalofit

In [26]:
def compute_sigma8_from_pk(k, pk):
    """Given a power spectrum P(k), compute sigma8."""
    R = 8.0
    x = k * R
    W = np.zeros(x.shape)
    m = x < 1.e-3
    W[m] = 1.0
    W[~m] =3.0 / x[~m]**3 * (np.sin(x[~m]) - x[~m] * np.cos(x[~m]))
    y = pk * W**2 * k**3
    sigma2 = scipy.integrate.simpson(y, x=np.log(x))
    sigma = np.sqrt(sigma2 / (2.0 * np.pi**2))

    return sigma

# Cosmological parameters
As = 2.105  # 10^9 A_s
h = 0.6766
Om = 0.3111
Ob = 0.02242 / h ** 2
ns = 0.9665
tau = 0.0561

# Define k integration range


def linear_sigma8(As, Om, Ob, h, ns):
    """Calculated from Deaglan's emulator."""
    return symbolic_pofk.linear.As_to_sigma8(As, Om, Ob, h, ns)
    # print('sigma8 from emulator', sigma8)

# # Test linear sigma8
# pk_lin = symbolic_pofk.linear.plin_emulated(k, sigma8, Om, Ob, h, ns,
#     emulator='fiducial', extrapolate=True)
# new_sigma8 = compute_sigma8(k, pk_lin)
# print('sigma8 from integral:', new_sigma8)

# Get non-linear sigma8
def nonlinear_sigma8(As, Om0, Ob, h, ns, ks):
    a = 1.
    sigma8 = linear_sigma8(As, Om0, Ob, h, ns)  # Linear sigma8
    pk_nl = syrenhalofit.run_halofit(
        ks, sigma8, Om, Ob, h, ns, a, emulator='fiducial', extrapolate=True,
        which_params='Bartlett', add_correction=True)
    return compute_sigma8_from_pk(ks, pk_nl)
# print('non-linear sigma8:', sigma8_nl)
# print('sigma8 non-linear bigger by a factor', sigma8_nl / sigma8)


In [18]:
kmin, kmax, nk = 1e-4, 1e1, 256
ks = np.logspace(np.log10(kmin), np.log10(kmax), nk) # Wavenumber

In [21]:
linear_sigma8(As, Om, Ob, h, ns)

0.825706107176727

In [20]:
nonlinear_sigma8(As, Om, Ob, h, ns, ks)

0.9203769215120938

In [25]:
 1e-10 * np.exp(3.0448)

2.1005829616811546e-09